In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import time
import os
import csv
from config import password
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

top1000_csv = "resources\mojotop1000.csv"
# driver = webdriver.Chrome()

IMDB_list_url = 'https://www.imdb.com/list/ls098063263/?page=1'

In [2]:
executable_path = {'executable_path':"C:/Program Files/Google/Chrome/Chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
movie_list = []
movie_dict = {}
for i in range(1,11):
    browser.visit(f'https://www.imdb.com/list/ls098063263/?page={i}')
    time.sleep(5)

    # Set HTML object
    html = browser.html

    # parse HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

   
    

    movie_title = soup.find_all('h3', class_="lister-item-header")
    movie_genre = soup.find_all('span', class_='genre')


    for (title, genre) in zip(movie_title, movie_genre):
        anchor = title.find('a')
        for b in anchor:
            movie_dict = {}
            movie_dict['Title'] = b
            movie_dict['Genre'] = genre.text[1:-12]
            movie_list.append(movie_dict)

    # movie_dict["title"] = b, movie_dict["genre"] = genre.text
# browser.links.find_by_partial_text('next').click()

# print(movie_list)
# print(movie_title)
# print(movie_genre)


In [4]:
movie_df = pd.DataFrame(movie_list)
movie_df

,Title,Genre
0,Avatar,"Action, Adventure, Fantasy"
1,Avengers: Endgame,"Action, Adventure, Drama"
2,Titanic,"Drama, Romance"
3,Star Wars: Episode VII - The Force Awakens,"Action, Adventure, Sci-Fi"
4,Avengers: Infinity War,"Action, Adventure, Sci-Fi"
...,...,...
995,The A-Team,"Action, Adventure, Thriller"
996,Tootsie,"Comedy, Drama, Romance"
997,In the Line of Fire,"Action, Crime, Drama"
998,Analyze This,"Comedy, Crime"


In [5]:
top1000 = pd.read_csv(top1000_csv)
top1000.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,1,Avatar,2847246203,760507625,0.27,2086738578,0.73,2009
1,2,Avengers: Endgame,2797501328,858373000,0.31,1939128328,0.69,2019
2,3,Titanic,2201647264,659363944,0.30,1542283320,0.70,1997
3,4,Star Wars: Episode VII - The Force Awakens,2069521700,936662225,0.45,1132859475,0.55,2015
4,5,Avengers: Infinity War,2048359754,678815482,0.33,1369544272,0.67,2018


In [6]:
merge_pd = top1000.merge(movie_df, how='left', on = 'Title')
merge_pd = merge_pd.astype({"Worldwide Lifetime Gross": str})
merge_pd = merge_pd[["Title","Genre","Worldwide Lifetime Gross", "Year"]]
merge_pd.head(100)
merge_pd

,Title,Genre,Worldwide Lifetime Gross,Year
0,Avatar,"Action, Adventure, Fantasy",2847246203,2009
1,Avengers: Endgame,"Action, Adventure, Drama",2797501328,2019
2,Titanic,"Drama, Romance",2201647264,1997
3,Star Wars: Episode VII - The Force Awakens,"Action, Adventure, Sci-Fi",2069521700,2015
4,Avengers: Infinity War,"Action, Adventure, Sci-Fi",2048359754,2018
...,...,...,...,...
1017,Evan Almighty,NaN,174440724,2007
1018,AmÃ©lie,NaN,174118254,2001
1019,Identity Thief,NaN,173965010,2013
1020,The Nutcracker and the Four Realms,NaN,173961069,2018


In [7]:
merge_pd[['Genre 1', 'Genre 2', 'Genre 3']] = merge_pd['Genre'].str.split(', ', expand=True)
merge_pd = merge_pd[["Title","Genre 1","Genre 2","Genre 3","Worldwide Lifetime Gross", "Year"]]
merge_pd.head(100)
merge_pd

,Title,Genre 1,Genre 2,Genre 3,Worldwide Lifetime Gross,Year
0,Avatar,Action,Adventure,Fantasy,2847246203,2009
1,Avengers: Endgame,Action,Adventure,Drama,2797501328,2019
2,Titanic,Drama,Romance,None,2201647264,1997
3,Star Wars: Episode VII - The Force Awakens,Action,Adventure,Sci-Fi,2069521700,2015
4,Avengers: Infinity War,Action,Adventure,Sci-Fi,2048359754,2018
...,...,...,...,...,...,...
1017,Evan Almighty,NaN,NaN,NaN,174440724,2007
1018,AmÃ©lie,NaN,NaN,NaN,174118254,2001
1019,Identity Thief,NaN,NaN,NaN,173965010,2013
1020,The Nutcracker and the Four Realms,NaN,NaN,NaN,173961069,2018


In [8]:
Genre1_pd = pd.DataFrame(merge_pd['Genre 1'].value_counts())
Genre1_pd = Genre1_pd.rename(columns={'Genre 1':'Genre'})
Genre1_pd["Genre Name"]=Genre1_pd.index
Genre1_pd.reset_index(inplace=True)
Genre1_pd.drop('index', axis=1, inplace=True)
Genre1_pd

,Genre,Genre Name
0,433,Action
1,152,Animation
2,138,Comedy
3,83,Adventure
4,71,Drama
5,31,Biography
6,29,Crime
7,21,Horror
8,3,Mystery
9,3,Fantasy


In [9]:
Genre2_pd = pd.DataFrame(merge_pd['Genre 2'].value_counts())
Genre2_pd = Genre2_pd.rename(columns={'Genre 2':'Genre'})
Genre2_pd["Genre Name"]=Genre2_pd.index
Genre2_pd.reset_index(inplace=True)
Genre2_pd.drop('index', axis=1, inplace=True)
Genre2_pd

,Genre,Genre Name
0,395,Adventure
1,152,Drama
2,81,Comedy
3,49,Romance
4,48,Crime
5,39,Mystery
6,34,Family
7,26,Thriller
8,25,Action
9,22,Fantasy


In [10]:
Genre3_pd = pd.DataFrame(merge_pd['Genre 3'].value_counts())
Genre3_pd = Genre3_pd.rename(columns={'Genre 3':'Genre'})
Genre3_pd["Genre Name"]=Genre3_pd.index
Genre3_pd.reset_index(inplace=True)
Genre3_pd.drop('index', axis=1, inplace=True)
Genre3_pd

,Genre,Genre Name
0,151,Comedy
1,130,Sci-Fi
2,114,Thriller
3,96,Fantasy
4,61,Drama
5,49,Family
6,47,Romance
7,33,Crime
8,24,Adventure
9,21,Mystery


In [11]:
Genre_total_pd = Genre3_pd.merge(Genre2_pd,on='Genre Name', how='outer')#.merge(Genre3_pd,on='Genre Name',how='outer')
Genre_total_pd
Genre_total_pd = Genre_total_pd.merge(Genre1_pd,on='Genre Name', how='outer')
# Genre_total_pd
Genre_total_pd.fillna(0)
# Genre_total_pd = (Genre3_pd + Genre2_pd + Genre1_pd).value_counts()
# Genre_total_pd

,Genre_x,Genre Name,Genre_y,Genre
0,151.0,Comedy,81.0,138.0
1,130.0,Sci-Fi,20.0,0.0
2,114.0,Thriller,26.0,0.0
3,96.0,Fantasy,22.0,3.0
4,61.0,Drama,152.0,71.0
5,49.0,Family,34.0,0.0
6,47.0,Romance,49.0,0.0
7,33.0,Crime,48.0,29.0
8,24.0,Adventure,395.0,83.0
9,21.0,Mystery,39.0,3.0


In [12]:
Genre_total_pd["Total count"] = Genre_total_pd.sum(axis=1)
Genre_total_pd

C:\Users\Aaron\AppData\Local\Temp\ipykernel_27344\3494757172.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Genre_total_pd["Total count"] = Genre_total_pd.sum(axis=1)


,Genre_x,Genre Name,Genre_y,Genre,Total count
0,151.0,Comedy,81.0,138.0,370.0
1,130.0,Sci-Fi,20.0,NaN,150.0
2,114.0,Thriller,26.0,NaN,140.0
3,96.0,Fantasy,22.0,3.0,121.0
4,61.0,Drama,152.0,71.0,284.0
5,49.0,Family,34.0,NaN,83.0
6,47.0,Romance,49.0,NaN,96.0
7,33.0,Crime,48.0,29.0,110.0
8,24.0,Adventure,395.0,83.0,502.0
9,21.0,Mystery,39.0,3.0,63.0


In [13]:
Sorted_totals_pd = (Genre_total_pd.sort_values(by="Total count", ascending=False))
Sorted_totals_pd = Sorted_totals_pd[["Genre Name","Total count"]]
Sorted_totals_pd

,Genre Name,Total count
8,Adventure,502.0
18,Action,458.0
0,Comedy,370.0
4,Drama,284.0
19,Animation,152.0
1,Sci-Fi,150.0
2,Thriller,140.0
3,Fantasy,121.0
7,Crime,110.0
6,Romance,96.0


In [14]:
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'Movies_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [15]:
engine.table_names()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_27344\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['genre_counts', 'top1000', 'merged']

In [16]:
top1000.to_sql(name='top1000', con=engine, if_exists='append', index=False)

1000

In [17]:
Sorted_totals_pd.to_sql(name='genre_counts', con=engine, if_exists='append', index=False)

21

In [18]:
merge_pd.to_sql(name='merged', con=engine, if_exists='append', index=False)

22